In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sn
%matplotlib inline

In [4]:
train=pd.read_csv("train.csv")
train_labels = pd.read_csv("train_labels.csv")
test=pd.read_csv("test.csv")
specs=pd.read_csv("specs.csv")
sample_submission =pd.read_csv("sample_submission.csv")

In [5]:
train.shape,train_labels.shape,test.shape,specs.shape,sample_submission.shape

((11341042, 11), (17690, 7), (1156414, 11), (386, 3), (1000, 2))

In [6]:
for k in train.columns:
    print(k,train[k].nunique())
    
train.head()

event_id 384
game_session 303319
timestamp 10777308
event_data 10893749
installation_id 17000
event_count 3368
event_code 42
game_time 834048
title 44
type 4
world 4


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [7]:
for k in specs.columns:
    print(k,specs[k].nunique())
    
specs.head()

event_id 386
info 168
args 191


,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [9]:
train.query('game_session == "45bb1e1b6b50c07b"')

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE


In [10]:
train.groupby('game_session').size().nlargest(5)

game_session
6e6e697f2e593de1    3368
bb1f09ec062b6660    3182
33495c8f126e2ef9    2505
34c82b23355e378c    2456
8fe0ab3c3e448a04    2398
dtype: int64

In [33]:
train.query('game_session == "8fe0ab3c3e448a04"').head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
6836702,d2278a3b,8fe0ab3c3e448a04,2019-08-12T17:24:05.473Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",994491d0,1,2000,0,Bottle Filler (Activity),Activity,MAGMAPEAK
6836703,b7530680,8fe0ab3c3e448a04,2019-08-12T17:24:05.740Z,"{""previous_jars"":[],""round"":1,""event_count"":2,...",994491d0,2,2020,178,Bottle Filler (Activity),Activity,MAGMAPEAK
6836704,67439901,8fe0ab3c3e448a04,2019-08-12T17:24:08.150Z,"{""description"":""Hello! Today, we're making san...",994491d0,3,3010,2770,Bottle Filler (Activity),Activity,MAGMAPEAK
6836705,bb3e370b,8fe0ab3c3e448a04,2019-08-12T17:24:21.671Z,"{""bottle"":{""amount"":1,""color"":""green""},""bottle...",994491d0,4,4030,16271,Bottle Filler (Activity),Activity,MAGMAPEAK
6836706,67439901,8fe0ab3c3e448a04,2019-08-12T17:24:22.684Z,"{""description"":""Look at it go! idle"",""identifi...",994491d0,6,3010,17305,Bottle Filler (Activity),Activity,MAGMAPEAK


In [ ]:
train.query('game_session == "8fe0ab3c3e448a04" or type == "Assessment"').head()

In [17]:
train['world'].unique()

array(['NONE', 'MAGMAPEAK', 'TREETOPCITY', 'CRYSTALCAVES'], dtype=object)

In [25]:
train['type'].unique()

array(['Clip', 'Activity', 'Game', 'Assessment'], dtype=object)

In [19]:
train_labels.head(15)

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3
5,197a373a77101924,0006c192,Cauldron Filler (Assessment),1,0,1.0,3
6,957406a905d59afd,0006c192,Bird Measurer (Assessment),1,1,0.5,2
7,b2297d292892745a,0006c192,Mushroom Sorter (Assessment),0,4,0.0,0
8,ae691ec5ad5652cf,00129856,Bird Measurer (Assessment),1,0,1.0,3
9,7b536271e99518f0,001d0ed0,Bird Measurer (Assessment),0,5,0.0,0


In [24]:
train_labels['title'].unique()

array(['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)',
       'Cart Balancer (Assessment)'], dtype=object)

In [20]:
test.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [34]:
train.groupby('type').head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK
5,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.279Z,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK
6,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.913Z,"{""coordinates"":{""x"":601,""y"":570,""stage_width"":...",0001e90f,5,4070,10622,Sandcastle Builder (Activity),Activity,MAGMAPEAK
362,6d90d394,f11eb823348bfa23,2019-09-06T18:00:51.915Z,"{""version"":""1.0"",""level"":0,""round"":0,""event_co...",0001e90f,1,2000,0,Scrub-A-Dub,Game,MAGMAPEAK
363,7040c096,f11eb823348bfa23,2019-09-06T18:00:56.121Z,"{""coordinates"":{""x"":1278,""y"":957,""stage_width""...",0001e90f,2,4010,4205,Scrub-A-Dub,Game,MAGMAPEAK
364,5a848010,f11eb823348bfa23,2019-09-06T18:00:57.986Z,"{""movie_id"":""scrubadub_intro"",""duration"":13766...",0001e90f,3,2080,6057,Scrub-A-Dub,Game,MAGMAPEAK


In [44]:
train.query('installation_id== "0006a69f" and type == "Assessment" and game_session=="901acc108f55a5a1" and event_code =="4100"')

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
2228,25fa8af4,901acc108f55a5a1,2019-08-06T05:22:32.357Z,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,44,4100,31011,Mushroom Sorter (Assessment),Assessment,TREETOPCITY


In [38]:
train_labels.query('installation_id== "0006a69f" and game_session =="901acc108f55a5a1"')

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3


In [39]:
train_labels.query('installation_id== "0006a69f"')

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [45]:
train.query('installation_id== "0006a69f" and type == "Assessment" and game_session=="77b8ee947eb84b4e" and event_code =="4110"')

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
2709,17113b36,77b8ee947eb84b4e,2019-08-06T05:35:54.898Z,"{""correct"":false,""caterpillars"":[11,8,3],""even...",0006a69f,29,4110,35771,Bird Measurer (Assessment),Assessment,TREETOPCITY
2715,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:01.927Z,"{""correct"":false,""caterpillars"":[11,8,11],""eve...",0006a69f,35,4110,42805,Bird Measurer (Assessment),Assessment,TREETOPCITY
2720,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:06.512Z,"{""correct"":false,""caterpillars"":[11,8,5],""even...",0006a69f,40,4110,47388,Bird Measurer (Assessment),Assessment,TREETOPCITY
2725,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:09.739Z,"{""correct"":false,""caterpillars"":[11,8,7],""even...",0006a69f,45,4110,50605,Bird Measurer (Assessment),Assessment,TREETOPCITY
2730,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:13.951Z,"{""correct"":false,""caterpillars"":[11,8,4],""even...",0006a69f,50,4110,54822,Bird Measurer (Assessment),Assessment,TREETOPCITY
2733,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:17.407Z,"{""correct"":false,""caterpillars"":[11,8,4],""even...",0006a69f,53,4110,58280,Bird Measurer (Assessment),Assessment,TREETOPCITY
2738,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:21.390Z,"{""correct"":false,""caterpillars"":[11,8,2],""even...",0006a69f,58,4110,62256,Bird Measurer (Assessment),Assessment,TREETOPCITY
2743,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:26.296Z,"{""correct"":false,""caterpillars"":[11,8,1],""even...",0006a69f,63,4110,67164,Bird Measurer (Assessment),Assessment,TREETOPCITY
2750,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:32.187Z,"{""correct"":false,""caterpillars"":[11,8,1],""even...",0006a69f,70,4110,73056,Bird Measurer (Assessment),Assessment,TREETOPCITY
2756,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:38.140Z,"{""correct"":false,""caterpillars"":[5,8,1],""event...",0006a69f,76,4110,79006,Bird Measurer (Assessment),Assessment,TREETOPCITY


In [46]:
train.query('installation_id== "0006a69f" and type == "Assessment" and game_session=="9501794defd84e4d" and event_code =="4100"')

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
3725,25fa8af4,9501794defd84e4d,2019-08-06T20:35:12.290Z,"{""correct"":false,""stumps"":[3,2,5],""event_count...",0006a69f,29,4100,18484,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
3734,25fa8af4,9501794defd84e4d,2019-08-06T20:35:16.846Z,"{""correct"":true,""stumps"":[2,3,5],""event_count""...",0006a69f,38,4100,23043,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
